In [ ]:
import branca
import geopandas as gpd
import intake
import ipywidgets as widgets
import pandas as pd

from IPython.display import Markdown, HTML

import setup_corridors_stats
from create_parallel_corridors import IMG_PATH, DATA_PATH
from shared_utils import map_utils, geography_utils
from shared_utils import calitp_color_palette as cp

catalog = intake.open_catalog("./*.yml")

In [ ]:
display(HTML("<h1>Parallel / Intersecting Bus Routes to the State Highway Network </h1>"))

In [ ]:
# Rename columns to better display
rename_dict = {
    "itp_id": "ITP ID",
    "pct_parallel": "% parallel",
    "unique_route_id": "# Unique Bus Routes",
    "num_parallel": "# Parallel Bus Routes",
}

In [ ]:
def display_operator_stats(operator_stats, select_col = "itp_id", operator_name = 182):
    operator_stats = operator_stats[operator_stats[select_col]==operator_name]
    # Style table
    operator_table = (operator_stats.sort_values("pct_parallel", ascending=False)
                      .rename(columns = rename_dict)
                      .style.format({'% parallel': '{:,.1%}'})
                      .hide(axis="index")
                      .to_html()
    )    
    
    display(HTML(f"<h2> Summary Stats for ITP ID: {operator_name}</h2>"))
    display(HTML(operator_table))

In [ ]:
def make_transit_map(operator_df, hwy_df):    
    def data_to_plot(df):
        keep_cols = ["itp_id", "route_id", 
                     "Route", "County", "District", "RouteType",
                     "pct_route", "pct_highway", "parallel",
                     "geometry"
                    ]
        df = df[keep_cols].reset_index(drop=True)
        df = df.assign(
            geometry = df.geometry.buffer(200).simplify(tolerance=100),
        )

        # Use simplify to make gdf smaller
        # folium map is creating too large of an HTML file to check in

        return df

    to_map = data_to_plot(operator_df)
    
    # Set various components for map
    hwys_popup_dict = {
        "Route": "Highway Route",
        "RouteType": "Route Type",
        "County": "County"   
    }

    transit_popup_dict = {
        "itp_id": "Operator ITP ID",
        "route_id": "Route ID",
        "pct_route": "% overlapping route",
        "pct_highway": "% overlapping highway",
    }

    hwys_color = branca.colormap.StepColormap(
        colors=["black", "gray"],
    )

    colorscale = branca.colormap.StepColormap(
        colors=[
            cp.CALITP_CATEGORY_BRIGHT_COLORS[0], #blue
            cp.CALITP_CATEGORY_BRIGHT_COLORS[1] # orange
        ],
    )
    
    # Instead of using county centroid, calculate centroid from transit_df
    # Otherwise, it's too zoomed out from where transit routes are
    transit_centroid = (to_map
                        .to_crs(geography_utils.WGS84).geometry.centroid
                        .iloc[0]
                       )

    LAYERS_DICT = {
        "Highways": {"df": hwy_df,
            "plot_col": "Route",
            "popup_dict": hwys_popup_dict, 
            "tooltip_dict": hwys_popup_dict,
            "colorscale": hwys_color,
        },
        "Transit Routes": {"df": to_map,
            "plot_col": "parallel",
            "popup_dict": transit_popup_dict, 
            "tooltip_dict": transit_popup_dict,
            "colorscale": colorscale,
        },
    }
    
    LEGEND_URL = (
        "https://github.com/cal-itp/data-analyses/raw/"
        "main/bus_service_increase/"
        "img/legend_intersecting_parallel.png"
    )
    
    LEGEND_DICT = {
        "legend_url": LEGEND_URL,
        "legend_bottom": 85,
        "legend_left": 5,
    }
     
    
    fig = map_utils.make_folium_multiple_layers_map(
        LAYERS_DICT,
        fig_width = 700, fig_height = 700, 
        zoom=11, 
        centroid = [round(transit_centroid.y,2), 
                    round(transit_centroid.x, 2)], 
        title=f"Parallel vs Intersecting Lines for {to_map.itp_id.iloc[0]}",
        legend_dict = LEGEND_DICT
    )
    
    display(fig)
    #fig.save(f"{IMG_PATH}parallel_{operator_name}.html")
    #print(f"{operator_name} map saved")

In [ ]:
def display_highway_stats(hwy_stats, select_col = "Route", hwy_name = 5):
    # Subset for highways
    hwy_stats = hwy_stats[hwy_stats[select_col]==hwy_name]
    hwy_table = (hwy_stats.sort_values("pct_parallel", ascending=False)
                 .rename(columns = rename_dict)
                 .reset_index(drop=True)
                 .style.format({'% parallel': '{:,.1%}'})
                 .hide(axis="index")
                 .to_html()
                )    
    
    display(HTML(f"<h2>Summary Stats for Highway Route: {hwy_name} </h2>"))
    display(HTML(hwy_table))
    

In [ ]:
def initial_data(gdf):
    operator_hwys = (gdf[["itp_id", "Route"]][gdf.Route.notna()]
                 .drop_duplicates()
                 .astype(int)
                )

    operator_hwys = (operator_hwys.groupby("itp_id")["Route"]
                     .apply(lambda x: x.tolist())
                     .reset_index()
                     .rename(columns = {"Route": "hwy_list"})
                    )
    
    # Merge on the list of highways for each operator (m:1)
    gdf = pd.merge(gdf[gdf.Route.notna()], 
                   operator_hwys, 
                   on = "itp_id", validate = "m:1")
    
    return gdf

In [ ]:
def make_aggregated_data(gdf):
    # Need the full dataset when making the map
    # Move aggregations to be done ahead of time, before dropdown
    operator_stats = (setup_corridors_stats.aggregate(gdf, by="operator")
                      .sort_values("pct_parallel", ascending=False)
                      .reset_index(drop=True)
                     )

    hwy_stats = (setup_corridors_stats.aggregate(gdf, by="highway")
                 .sort_values("pct_parallel", ascending=False)
                 .reset_index(drop=True)
                )  
    
    return operator_stats, hwy_stats


def highways_for_operator(operator_stats, highways):
    # Select the highways for the map when plotting transit map
    highways_df = highways[highways.Route.isin(operator_stats.hwy_list.iloc[0])]
    return highways_df

In [ ]:
# Generalize selection column in ipywidget dropdown
# Be able to select operator or highway route, etc
def interactive_widget(select_col):
    # Cache the GCS download?
    # catalog entry might take awhile for ipywidget to spin up
    gdf = catalog.parallel_or_intersecting_routes.read()
    highways = catalog.highways_cleaned.read()
    
    gdf = initial_data(gdf)
    operator_stats, hwy_stats = make_aggregated_data(gdf)
    
    dropdown_labels = {
        "itp_id": "ITP ID",
        "Route": "Hwy Route",
    }
    
    dropdown = widgets.Dropdown(
        description=f"{dropdown_labels[select_col]}",
        options=gdf[select_col].astype(int).sort_values().unique().tolist(),
    )
    output = widgets.Output()

    display(dropdown)
    display(output)

    def on_selection(*args):
        output.clear_output()
        with output:
            if select_col=="Route":
                display_highway_stats(hwy_stats, select_col, dropdown.value)
            elif select_col=="itp_id":
                # Grab the full df and subset to specific
                # Break out the map-making function so the relevant highways can be extracted
                operator_df = gdf[gdf[select_col] == dropdown.value]
                hwy_df = highways_for_operator(operator_df, highways)
                
                display_operator_stats(operator_stats, select_col, dropdown.value)
                make_transit_map(operator_df, hwy_df)

    dropdown.observe(on_selection, names="value")
    on_selection()

In [ ]:
display(HTML("<h3>Parallel Corridor Stats by Operator </h3>"))
interactive_widget("itp_id")

In [ ]:
display(HTML("<h3>Parallel Corridor Stats by Highway Route</h3>"))
interactive_widget("Route")